In [2]:
import pandas as pd
import numpy as np
import os
from plotnine import *
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")


### Ingrese aquí el nombre de los integrantes del grupo
- Nombre integrante 1
- 
- 
- 

# Descripción del problema y la data
La data acompañada corresponde a clientes de una cadena de ropa, y su comportamiento con respecto a campañas de marketing que se le envian.  La `data` contiene 16305 clientes (filas), y 44 *features* (columnas), indicando su número de visitas a la tienda, las ventas, porcentaje de compras en los distintos items, montos gastados en distintos locales de la tienda y en distintos rangos de meses, etc. A estos clientes se les envió una campaña de marketing directa, ante la cual ellos respondieron (`target` = 1) o no lo hicieron (`target`=0).

Hacer una campaña de marketing directo a estos clientes puede tener un impacto importante en las ventas. Se estima que cada cliente que responde a estas campañas genera un profit de $\$28.40$ para la empresa. Sin embargo, hacer estas campañas tiene un costo estimado de $\$2.00$ por cliente.  Es por esto, que la empresa desea identificar correctamente aquellos clientes que son mas proclives a responder a este tipo de campaña de marketing.

El objetivo de este trabajo entonces es construir un **modelo de clasificación**, que permita predecir si un cliente va a responder o no a una campaña de marketing.



Para esto:
- Decida si utilizará todos los features o no. Puede eliminar features, o agregar nuevos si lo desea.  Explique en este documento lo que hizo al respecto.
- Transforme la data si es necesario. Note que los features tienen distintas escalas (por ejemplo, algunos son binarios (0,1), otros son porcentajes [0-1], otros son días [0-365], y otros montos gastados [0-22511].  Explicite **claramente** la o las transformaciones necesarias a la data. Su modelo será evaluado con nueva data, por lo que es necesario tener claro las transformaciones necesarias para poder aplicarlo. 
- Escoga una (o más de una) métrica que utilizará para poder evaluar el performance de sus modelos. Indique cuál será la métrica prioritaria para su evaluación, y el por qué de esta decisión.
- Escoga entre al menos 3 modelos de clasificación estudiados en el curso. Para cada uno de ellos, realice una optimización de hiper-parámetros, para escoger los mejores parámetros para cada uno de ellos.  **Atención**: al menos unos de estos tres modelos debe poder ser interpretable.
- Compare estos tres (o mas) modelos escogidos (con los mejores parámetros que optimizó) entre ellos, usando cross-validation.  Decida cuál es el mejor modelo para usted. **Agregue este modelo al final de este documento** (ver ejemplo).
- Utilice el mejor modelo obtenido pero que tenga capacidad de interpretación de los resultados, y analice este mismo. ¿Cuáles son las características o features más importantes para que un cliente responda positivamente a una campaña de márketing directo?

In [5]:

def one_hot_encoder(df):
  """
  DOCSTRING
  """
  chunks = []
  chunks.append(df)
  bool_cols = df.select_dtypes(include= [bool]).columns
  
  for col in bool_cols:
    chunk = pd.get_dummies(data[col], prefix=col)
    chunks.append(chunk)
  
  df = pd.concat(chunks, axis= 1)
  df.drop(bool_cols, axis= 1, inplace= True)
  

  return df



def get_best_features(X, y, models, cv):
  
  
  my_custom_score = make_scorer(my_scorer, greater_is_better=True)

  for model in models:
    
    pipeline = Pipeline([('ss', StandardScaler()),
                    ('', model())])

    gs = GridSearchCV(
                      pipeline,
                      param_grid=[models[model]],
                      scoring= my_custom_score,
                      cv=cv
    )
    
    gs.fit(X, y)
    
    print(f'model: {model.__name__}')
    print(f'\t best params: {gs.best_params_}')
    print(f'\t utilidad de la muestra: {gs.best_score_}')
  
  return 



def my_scorer(y_true, y_predicted):
  try:
    tn, fp, fn, tp = confusion_matrix(y_true, y_predicted).ravel()
    return (tp * 28.4) - (fp * 2)
  except:
    return 0

    


In [7]:
# Carga de Data
data = pd.read_csv('Clothstore_data.csv')
target = pd.read_csv('Clothstore_target.csv')
features  = one_hot_encoder(data)
features.shape

(16305, 47)

In [9]:
params = {
  RandomForestClassifier:{
    '__n_estimators': [30,50,100,150,200],
    '__max_depth':[5,7,10],
    '__min_samples_split': [2,5],
    '__class_weight': ['balanced_subsample', 'balanced']
    },
  SVC: {
    '__class_weight':['balanced'],
    '__kernel':['rbf','linear'],
    '__C':[0.1,1,10]
  },
  KNeighborsClassifier:{
    '__n_neighbors': [4,5,7,10]
  }
}

get_best_features(features,target, params, 3)


model: RandomForestClassifier
	 best params: {'__class_weight': 'balanced', '__max_depth': 5, '__min_samples_split': 2, '__n_estimators': 30}
	 utilidad de la muestra: 16567.066666666666
model: SVC
	 best params: {'__C': 10, '__class_weight': 'balanced', '__kernel': 'linear'}
	 utilidad de la muestra: 19094.8
model: KNeighborsClassifier
	 best params: {'__n_neighbors': 5}
	 utilidad de la muestra: 5708.933333333333


In [ ]:
# best_params = {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200, 'class_weight':'balanced'}

# rf = RandomForestClassifier()
# rf.set_params(**best_params)
# rf.fit(features, target)
# temp_df = list(zip(features.columns, rf.feature_importances_))
# temp_df = pd.DataFrame(temp_df, columns =['Feature', 'Importance'])
# top_feature_importance = temp_df.sort_values(by= 'Importance', ascending= False).iloc[0:20,[0]]
# top_feature_importance = list(top_feature_importance["Feature"])
# top_feature_importance

In [ ]:
# params = {
#   RandomForestClassifier:{
#     '__n_estimators': [10,15,20,25,30,50,100,200,300],
#     '__max_depth':[5,7,10,15,20,30,50,100,200],
#     '__min_samples_split': [2, 5, 7, 10],
#     '__class_weight':['balanced','balanced_subsample']
#     },
#   SVC : {
#     '__C': [0.1, 1, 10, 20, 50, 100, 1000],
#     '__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#     '__kernel': ['rbf']
#     },
#   KNeighborsClassifier : {
#     '__n_neighbors': [2,3,4,5,6,7,9,9,10]
#     }
# }

# get_best_features(features[top_feature_importance],target, params, 3)


In [26]:

# Mejor modelo
best_params =  {'C': 10, 'class_weight': 'balanced', 'kernel': 'linear'}

model = SVC()

kf = KFold(n_splits= 10, shuffle = True)

results = {
  'f1':[],
  'precision':[],
  'recall': [],
  'utilidad_muestra':[]
}


for train, test in kf.split(features):
  
  features_selected = features
  
  #scaler = StandardScaler()
  scaler = MinMaxScaler()
  scaler.fit(features_selected.iloc[train,:])
  
  clf = model
  clf.set_params(**best_params)
  clf.fit(scaler.transform(features_selected.iloc[train,:]), target.iloc[train,:])
  preds = clf.predict(scaler.transform(features_selected.iloc[test,:]))
  
  results['f1'].append(f1_score(target.iloc[test,:], preds))
  results['precision'].append(precision_score(target.iloc[test,:], preds))
  results['recall'].append(recall_score(target.iloc[test,:], preds))
  results['utilidad_muestra'].append(my_scorer(target.iloc[test,:], preds))

results = pd.DataFrame(results)

for col in results:
  print(f'avg for {col} is: {results[col].mean()}')
  print(f'std for {col} is: {results[col].std()}')




avg for f1 is: 0.47409240043606626
std for f1 is: 0.01809936823726951
avg for precision is: 0.3257434984610597
std for precision is: 0.01663275512432511
avg for recall is: 0.8721831165687544
std for recall is: 0.017119383019377942
avg for utilidad_muestra is: 5730.48
std for utilidad_muestra is: 413.949768289181


In [27]:
print(f'Escenario base: {(2708 * 28.4) - (16305-2708) * 2}')
print('Resultados SVM:',((2708 * pd.Series(results['recall']).mean()) * 28.4) - ((16305-2708)*(1- pd.Series(results['precision']).mean()))*0.2)


Escenario base: 49713.2
Resultados SVM: 65243.58825229151


In [28]:

# Mejor modelo interpretable
best_params =  {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 30}

model = RandomForestClassifier()

kf = KFold(n_splits= 10, shuffle = True)

results = {
  'f1':[],
  'precision':[],
  'recall': [],
  'utilidad_muestra':[]
}


for train, test in kf.split(features):
  
  features_selected = features
  
  #scaler = StandardScaler()
  scaler = MinMaxScaler()
  scaler.fit(features_selected.iloc[train,:])
  
  clf = model
  clf.set_params(**best_params)
  clf.fit(scaler.transform(features_selected.iloc[train,:]), target.iloc[train,:])
  preds = clf.predict(scaler.transform(features_selected.iloc[test,:]))
  
  results['f1'].append(f1_score(target.iloc[test,:], preds))
  results['precision'].append(precision_score(target.iloc[test,:], preds))
  results['recall'].append(recall_score(target.iloc[test,:], preds))
  results['utilidad_muestra'].append(my_scorer(target.iloc[test,:], preds))

results = pd.DataFrame(results)

for col in results:
  print(f'avg for {col} is: {results[col].mean()}')
  print(f'std for {col} is: {results[col].std()}')


avg for f1 is: 0.4871006494553396
std for f1 is: 0.012800071791439988
avg for precision is: 0.36553031700658234
std for precision is: 0.01258466638416254
avg for recall is: 0.7306193486570745
std for recall is: 0.02194289292975946
avg for utilidad_muestra is: 4928.68
std for utilidad_muestra is: 290.8084088651265


In [29]:
print(f'Escenario base: {(2708 * 28.4) - (16305-2708) * 2}')
print('Resultados Random Forest:',((2708 * pd.Series(results['recall']).mean()) * 28.4) - ((16305-2708)*(1- pd.Series(results['precision']).mean()))*0.2)


Escenario base: 49713.2
Resultados Random Forest: 54464.51151510705


In [37]:
best_params =  {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 30}

rf = RandomForestClassifier()
rf.set_params(**best_params)
rf.fit(features, target)
temp_df = list(zip(features.columns, rf.feature_importances_))
temp_df = pd.DataFrame(temp_df, columns =['Feature', 'Importance'])
temp_df_sorted = temp_df.sort_values(by= 'Importance', ascending= False)

import plotly.express as px
fig = px.bar(temp_df_sorted, x='Feature', y='Importance')
fig.show()

# RESULTADO FINAL MEJOR MODELO
En la siguiente celda, escriba el mejor modelo encontrado por usted, de forma que el profesor y su ayudante puedan evaluar el performance de este modelo escogido ante nuevos datos.  Ponga por lo tanto todos los pasos requeridos para poder hacer esto, incluyendo el preprocesing que sea necesario.

**Ejemplo**: si el modelo final fuera aplicar KNN con 8-vecinos usando los datos normalizados y sin la columna `has_valid_phone`, entonces debería ser algo así:


In [93]:

(2708 * 28.4) - (16305-2708) * 2
((2708 * 0.87) * 28.4) -   ((16305-2708)*(1-0.32))*0.2




65060.07199999999